In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException,ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


In [41]:
#Parámetros
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
options.add_argument("-inprivate")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [87]:
def data_walmart(zip_code,busqueda):
    busqueda=busqueda.replace(" ","+")

    #Cambia ubicación para obtener precios
    url="https://super.walmart.com.mx/"
    url_busqueda_template="https://super.walmart.com.mx/search?q="+busqueda+"&page={}"
    driver=webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(5)
    try:
        elemento=driver.find_element(By.NAME,"select-store-button")
    except NoSuchElementException:
        elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')
    elemento.click()
    #Ingresa código postal
    time.sleep(4)
    cp=driver.find_element(By.CSS_SELECTOR,"input[data-automation-id='store-zip-code']")
    cp.clear()
    cp.send_keys(zip_code)
    time.sleep(4)
    #Obtener datos de la tienda
    tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
    address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
    radio_buttons=driver.find_element(By.NAME,'pickup-store')
    #Aceptar elección
    elegir=driver.find_element(By.CSS_SELECTOR,"button[data-automation-id='save-label']")
    elegir.click()
    time.sleep(4)

    all_data=[]
    for page_number in range(1,4):
        url_busqueda=url_busqueda_template.format(page_number)
        driver.get(url_busqueda)
        time.sleep(4)
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        results = soup.find_all('div', {'data-stack-index': '0'})
        span_tags = []
        product_titles = []
        # Loop para obtener precios y productos
        for result in results:
            span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
            product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

        precios = []

        for span in span_tags:
            text = span.text
            if 'precio actual' in text:
                precios.append(text.split("precio actual")[1].strip())
        #Fecha y hora de la consulta
        fecha=time.strftime("%d/%m/%y")
        hora=time.strftime("%H:%M:%S")
        fecha_hora=hora+" "+fecha
        dfx=pd.DataFrame({'producto':product_titles,'precio':precios,'tienda':tienda,'direccion':address,'busqueda':busqueda,'fecha_consulta':fecha_hora})
        all_data.append(dfx)
        time.sleep(5)
    df=pd.concat(all_data,ignore_index=True)
    #Limpiar precios
    #Dejar solo números utilizando expresiones regulares
    df['precio'] = df['precio'].str.replace(r'[^0-9.]', '', regex=True)
    #Eliminar comas
    df['precio']=df['precio'].str.replace(',','',regex=True)
    #Convertir a numérico
    df['precio']=pd.to_numeric(df['precio'])
    df['producto']=df.producto.str.lower()
    return df
    


In [91]:
df=data_walmart("62170","limon")
print("registros: ",df.shape[0])
df.head(10)

registros:  134


,producto,precio,tienda,direccion,busqueda,fecha_consulta
0,limón sin semilla por kilo,34.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
1,limón agrio por kilo,34.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
2,limón eureka por kilo,52.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
3,limón agrio extra en malla 1.3 kg,60.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
4,naranja importada por kilo,26.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
5,limón sin semilla en malla 800 g,36.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
6,limón agrio en malla 1 kilo,37.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
7,té mccormick limón 27 sobres,25.0,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
8,té de limón por pieza,12.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23
9,naranja por kilo,26.9,Tlalpan,"Calzada De Tlalpan No. 1037, Colonia Américas ...",limon,18:48:14 22/08/23


In [43]:
# url="https://super.walmart.com.mx/"
# driver=webdriver.Edge(options=options)
# driver.get(url)
# time.sleep(5)

In [44]:
# try:
#     elemento = driver.find_element(By.NAME, 'select-store-button')
    
# except NoSuchElementException:
#     elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')

# elemento.click()

In [45]:
# #Código postal
# time.sleep(4)
# cp=driver.find_element(By.CSS_SELECTOR,'input[data-automation-id="store-zip-code"]')
# cp.clear()
# cp.send_keys('58090')
# time.sleep(4)
# tienda = driver.find_element(By.CSS_SELECTOR, 'span.b.f5.lh-copy.dark-gray').text
# address = driver.find_element(By.CSS_SELECTOR, 'span.f6[aria-label]').get_attribute('aria-label')
# radio_buttons = driver.find_element(By.NAME,'pickup-store')
# radio_buttons.click()
# #Elegir
# elegir=driver.find_element(By.CSS_SELECTOR,'button[data-automation-id="save-label"]')
# elegir.click()

In [46]:
# time.sleep(5)
# url_busqueda="https://super.walmart.com.mx/search?q=atún+en+lata"
# driver.get(url_busqueda)
# time.sleep(5)
# html=driver.page_source


In [47]:
# soup = BeautifulSoup(html, 'html.parser')

In [48]:
# results = soup.find_all('div', {'data-stack-index': '0'})

In [49]:
# span_tags = []
# product_titles = []

# # Loop through each individual result and select the desired elements
# for result in results:
#     span_tags.extend(result.select('div[data-automation-id="product-price"] span'))
#     product_titles.extend([title.text for title in result.select('span[data-automation-id="product-title"]')])

# precios = []

# for span in span_tags:
#     text = span.text
#     if 'precio actual' in text:
#         precios.append(text.split("precio actual")[1].strip())

In [50]:
# len(product_titles), len(precios)

(44, 44)

# #Llevar product_title y precios a un dataframe
# df=pd.DataFrame({'producto':product_titles,'precio':precios})
# df